In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
# !pip install selenium
# !pip install webdriver_manager
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import os
import glob
import zipfile
import pandas as pd
# 日本語の混じったcsvファイルをデータフレームに読み込むときに生じるエラーを抑える
import codecs   
# import csv


In [9]:
working_dir = '/content/drive/My Drive/05_決算書/'
os.chdir(working_dir)
data_dir = os.path.join(working_dir, 'python')   # 生データ用のフォルダ名を入れる
os.makedirs(data_dir, exist_ok=True)
os.chdir(data_dir)

In [ ]:
def edinet_code_dl():
  # seleniumでchromeからzipファイルをダウンロード
  chromeOptions = webdriver.ChromeOptions()
  chromeOptions.add_experimental_option(
      'prefs',
      {'download.default_directory':data_dir} # 保存先のディレクトリを指定
  )

  # ブラウザを非表示にする
  chromeOptions.add_argument('--headless')
  url = "https://disclosure.edinet-fsa.go.jp/E01EW/BLMainController.jsp"
  url += "?uji.bean=ee.bean.W1E62071.EEW1E62071Bean&uji.verb"
  url += "=W1E62071InitDisplay&TID=W1E62071&"
  url += "PID=W0EZ0001&SESSIONKEY=&lgKbn=2&dflg=0&iflg=0"

  driver = webdriver.Chrome(
      ChromeDriverManager().install(),
      options = chromeOptions,
  )
  driver.maximize_window()

  # EDINETコードリストにアクセス
  driver.get(url)
  driver.execute_script(
      "EEW1E62071EdinetCodeListDownloadAction('lgKbn=2&dflg=0&iflg=0&dispKbn=1');"
  )
  time.sleep(2)
  driver.quit()

edinet_code_dl()

In [54]:
def unzip_to_csv():
  edinet_zip = glob.glob('Edinetcode*.zip')[0]
  zip_file = zipfile.ZipFile(edinet_zip)
  zip_file.extractall('./')
  zip_file.close()

  # os.remove(edinet_zip)
  csv_path = glob.glob('EdinetcodeDlInfo.csv')[0]
  with open(csv_path, encoding='cp932') as f_read:
    new_path = 'EdinetcodeDlIinfo_utf8.csv'
    with open(new_path, 'w', encoding='utf-8') as f_out:
      f_out.write(f_read.read())
  
  csv_path = 'EdinetcodeDlIinfo_utf8.csv'
  return csv_path

edinet_csv = unzip_to_csv()
df = pd.read_csv(edinet_csv, header=1)
df.drop(columns=["提出者名（英字）", "提出者名（ヨミ）", "所在地"], inplace=True)

Index(['ＥＤＩＮＥＴコード', '提出者種別', '上場区分', '連結の有無', '資本金', '決算日', '提出者名', '提出者名（英字）',
       '提出者名（ヨミ）', '所在地', '提出者業種', '証券コード', '提出者法人番号'],
      dtype='object')


,ＥＤＩＮＥＴコード,提出者種別,上場区分,連結の有無,資本金,決算日,提出者名,提出者名（英字）,提出者名（ヨミ）,所在地,提出者業種,証券コード,提出者法人番号
0,E00004,内国法人・組合,上場,有,1491.0,5月31日,カネコ種苗株式会社,"KANEKO SEEDS CO., LTD.",カネコシュビョウカブシキガイシャ,前橋市古市町一丁目５０番地１２,水産・農林業,13760.0,5.070001e+12
1,E00006,内国法人・組合,上場,有,13500.0,5月31日,株式会社 サカタのタネ,SAKATA SEED CORPORATION,カブシキガイシャ サカタノタネ,横浜市都筑区仲町台２－７－１,水産・農林業,13770.0,6.020001e+12
2,E00007,内国法人・組合,上場,有,100.0,3月31日,株式会社雪国まいたけ,"YUKIGUNI MAITAKE CO.,LTD.",カブシキカイシャユキグニマイタケ,南魚沼市余川８９番地,水産・農林業,13750.0,1.010001e+12
3,E00008,内国法人・組合,上場,有,5500.0,3月31日,ホクト株式会社,HOKUTO CORPORATION,ホクトカブシキガイシャ,長野市南堀１３８番地１,水産・農林業,13790.0,6.100001e+12
4,E00009,内国法人・組合,上場,有,452.0,6月30日,株式会社アクシーズ,"AXYZ Co., Ltd",カブシキガイシャアクシーズ,鹿児島市草牟田二丁目１番８号,水産・農林業,13810.0,5.340001e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10144,E37832,個人（組合発行者を除く）,NaN,NaN,NaN,NaN,朝日 重紀,NaN,アサヒ シゲノリ,NaN,個人（組合発行者を除く）,NaN,NaN
10145,E37835,個人（組合発行者を除く）,NaN,NaN,NaN,NaN,大髙 功,NaN,オオタカ イサオ,NaN,個人（組合発行者を除く）,NaN,NaN
10146,E37839,外国法人・組合（有価証券報告書等の提出義務者以外）,NaN,NaN,0.0,NaN,エルシー・ファンド・エイト・エルピー,NaN,エルシーファンドエイトエルピー,ケイマン諸島、ＫＹ１－１１０４、グランドケイマン、アグランドハウス、私書箱３０９、メイプルズ...,外国法人・組合（有価証券報告書等の提出義務者以外）,NaN,NaN
10147,E37845,個人（組合発行者を除く）,NaN,NaN,NaN,NaN,相川 佳之,NaN,アイカワ ヨシユキ,NaN,個人（組合発行者を除く）,NaN,NaN
